In [ ]:
# Josh Kreutz
# 12/05/2024 
# AMATH 581 HW #6

import numpy as np
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.linalg import kron

# Parameters
L = 10
n = 64
beta = 1.0
D1 = D2 = 0.1
m = 1
tspan = np.arange(0, 4.5, 0.5)

# Spatial grid
x2 = np.linspace(-L, L, n + 1)
y2 = np.linspace(-L, L, n + 1)
x = x2[:n]
y = y2[:n]
X, Y = np.meshgrid(x, y)

# Initial conditions
u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))

u_hat = fft2(u)
v_hat = fft2(v)
a0 = np.hstack((u_hat.flatten(), v_hat.flatten()))

# Wave numbers
kx = (np.pi / L) * np.concatenate((np.arange(0, n / 2), np.arange(-n / 2, 0)))
ky = (np.pi / L) * np.concatenate((np.arange(0, n / 2), np.arange(-n / 2, 0)))
KX, KY = np.meshgrid(kx, ky)
K2 = KX**2 + KY**2

# Function for spectral RHS
def spc_rhs(t, u_v_vec, n, K2, D1, D2, beta):
    u_hat = u_v_vec[:n**2].reshape((n, n))
    v_hat = u_v_vec[n**2:].reshape((n, n))
    u = ifft2(u_hat)
    v = ifft2(v_hat)
    A2 = u**2 + v**2
    lambda_A = 1 - A2
    omega_A = -beta * A2

    u_rhs = lambda_A * u - omega_A * v + D1 * ifft2(-K2 * u_hat)
    v_rhs = lambda_A * v + omega_A * u + D2 * ifft2(-K2 * v_hat)

    u_rhs_hat = fft2(u_rhs).flatten()
    v_rhs_hat = fft2(v_rhs).flatten()
    
    return np.hstack((u_rhs_hat, v_rhs_hat))

# Solve PDE in spectral space
solution = solve_ivp(
    spc_rhs, [tspan[0], tspan[-1]], a0,
    args=(n, K2, D1, D2, beta), t_eval=tspan, method="RK45"
)
A1 = solution.y

# Chebyshev function
def cheb(N):
    if N == 0:
        D = np.zeros((1, 1))
        x = np.array([1.0])
    else:
        n = np.arange(0, N + 1)
        x = np.cos(np.pi * n / N).reshape(N + 1, 1)
        c = np.hstack(([2], np.ones(N - 1), [2])) * (-1) ** n
        X = np.tile(x, (1, N + 1))
        dX = X - X.T
        D = (np.outer(c, 1 / c)) / (dX + np.eye(N + 1))
        D -= np.diag(np.sum(D.T, axis=0))
    return D, x.reshape(N + 1)

# Chebyshev setup
N = 30
D, x = cheb(N)
D[-1, :] = 0
D[0, :] = 0
D_xx = np.dot(D, D) / (20 / 2) ** 2
y = x
I = np.eye(len(D_xx))
L = kron(I, D_xx) + kron(D_xx, I)

X, Y = np.meshgrid(x * 10, y * 10)
u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
uv0 = np.hstack([u.flatten(), v.flatten()])

# Reaction-Diffusion RHS
def RD_2D(t, uv, N, beta, D1, D2, L):
    N2 = (N + 1) ** 2
    u_flat = uv[:N2]
    v_flat = uv[N2:]
    A2 = u_flat**2 + v_flat**2
    lam = 1 - A2
    omega = -beta * A2

    rhs_u = D1 * L @ u_flat + lam * u_flat - omega * v_flat
    rhs_v = D2 * L @ v_flat + omega * u_flat + lam * v_flat

    return np.hstack([rhs_u, rhs_v])

# Solve Reaction-Diffusion system
sol = solve_ivp(
    RD_2D, [tspan[0], tspan[-1]], uv0,
    t_eval=tspan, args=(N, beta, D1, D2, L), method='RK45'
)

A2 = sol.y

# Results
print(A1)
print(A2)
print(f"A1 shape: {A1.shape}")
print(f"A2 shape: {A2.shape}")


[[ 24.94003847+0.00000000e+00j  12.73268299+2.33082011e-15j
   -1.38095598+7.29245665e-15j ... -64.02389647-3.42803098e-14j
  -67.76356741-3.81099223e-14j -61.18058974-3.04266028e-14j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j
   37.7389